## Sample Notebook - Python Pandas and some docker triggers

How to start: [01-SimpleDockerTest.ipynb](https://github.com/6za/runbooks-jupyter/blob/master/01-SimpleDockerTest.ipynb)

Sample Notenook mixing pandas and command line goodies.

In [94]:
import pandas as pd
print(pd.__version__)

0.25.3


Sample `hosts.csv`:

```txt
hostname,ip,arch,user
pi-node1,11.22.33.44,armv7l,pi
```

> File content list of names, ips and some annotations. 

In [95]:
hosts = pd.read_csv("hosts.csv")
supressed_columns = ['ip','user']

### List hostnames

In [96]:
hosts.drop(columns=supressed_columns)

,hostname,arch
0,pi-node1,unknown
1,pi-node2,armv7l
2,pi-node3,armv7l
3,kx-Lenovo-H520g,x86_64
4,nuc-01,x86_64
5,nuc-02,x86_64
6,nuc-03,x86_64
7,pi-node4,armv7l
8,pi-node5,armv7l
9,pi-node6,armv7l


In [97]:
hosts['ip'].to_csv(r'ip.txt', header=False, index=None, sep=' ')

In [98]:
!cat env.sh

#!/bin/bash
export DOCKER_TLS_VERIFY="1"
export DOCKER_CERT_PATH="/root/certs"
export DOCKER_HOST="tcp://$demo_docker_host:2376"

### List containers running on hosts

In [99]:
%%bash 
#source ./env.sh && docker ps
echo "ip|id|image|ports" > containers.csv
while read ip; do
    source ./env.sh 
    export DOCKER_HOST="tcp://$ip:2376"
    docker ps --format "$ip|{{.ID}}|{{.Image}}|{{.Ports}}" >> containers.csv 2>/dev/null || :
done <ip.txt

In [100]:
containers = pd.read_csv("containers.csv", sep = '|')
pd.merge(containers, hosts, on='ip', how='inner').drop(columns=supressed_columns)

,id,image,ports,hostname,arch
0,a0c35c3cdd5b,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,pi-node2,armv7l
1,61152c7c45f9,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,pi-node3,armv7l
2,52abb7d6236d,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,kx-Lenovo-H520g,x86_64
3,6ab17af75c27,amkay/sensor-exporter,0.0.0.0:9255->9255/tcp,kx-Lenovo-H520g,x86_64
4,3e719b0fad03,prom/prometheus,NaN,kx-Lenovo-H520g,x86_64
5,9ea5ac9a3688,grafana/grafana,NaN,kx-Lenovo-H520g,x86_64
6,1b947aee9a12,registry:2,0.0.0.0:5000->5000/tcp,kx-Lenovo-H520g,x86_64
7,634074457f5d,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,nuc-01,x86_64
8,d4748fcbce24,amkay/sensor-exporter,0.0.0.0:9255->9255/tcp,nuc-01,x86_64
9,f1aa6787c055,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,nuc-02,x86_64
